https://mp.weixin.qq.com/s?__biz=MzI0OTQyNzEzMQ==&mid=2247487769&idx=1&sn=a28b2c32bf6ef1e144b11a2ed59395e3&chksm=e990f4cddee77ddbdc57b0f99ca9d02556a28bf15f99b7c4967e289f67f4b737c8d156efc8ce&mpshare=1&scene=1&srcid=1022Silli2XRC2WLhXce6gOE#rd

首先要区别多分类 和多标签的区别。

* 多分类问题：一个样本的类别是多个，比如判别一个未知水果,它可能是苹果，也可能是香蕉，也可能是西瓜。水果的种类的是多样的，但是一个样本只可能是其中一类。  
* 多标签问题：一个样本包含多个类别。例如评价某个人和善，聪明，懒惰，即这个样本包含多个类别标签，既可以是Ａ也可以是Ｂ,类别之间不互斥。

    
多标签分类的适用场景较为常见，比如，一份歌单可能既属于标签旅行也属于标签驾车。有别于多分类分类，多标签分类中每个标签不是互斥的。多标签分类算法大概有两类流派：

* 采用One-vs-Rest（或其他方法）组合多个二分类基分类器；

* 改造经典的单分类器，比如，AdaBoost-MH与ML-KNN。

一些多分类数据集https://www.csie.ntu.edu.tw/~cjlin/libsvmtools/datasets/multilabel.html

### One-vs-Rest

基本思想：为每一个标签yi构造一个二分类器，正样本为含有yi标签的实例，负样本为不含有yi标签的实例；最后组合N个二分类器结果得到N维向量，可视作为在多标签上的得分

### ML-KNN

ML-KNN (multi-label K nearest neighbor)基于KNN算法，已知K近邻的标签信息，通过最大后验概率（Maximum A Posteriori）估计实例t是否应打上标签ℓ，  

https://www.jianshu.com/p/040f5e43a7de

算法原理

改进版knn ：ml-knn （multi-label knn）

1.通过knn 算法寻找和样本最近的Ｋ个样本  
2.统计ｋ个样本中每个类别的个数   
3.根据第二步的统计，采用 native bayes算法计算每个标签的概率  
计算每个样本的Ｋ个最近邻中，在样本标签为L的条件下，E^l_j(K个样本中有ｊ个Ｌ标签的样本数量)的概率。以及在样本不是标签Ｌ的条件下，E^l_j的概率  
4.输出类别概率

In [1]:
import numpy as np
from sklearn import metrics
from sklearn.datasets import load_svmlight_file
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.preprocessing import MultiLabelBinarizer

In [3]:
# load svm file
X_train, y_train = load_svmlight_file('data/tmc2007_train.svm', dtype=np.float64, multilabel=True)
X_test, y_test = load_svmlight_file('data/tmc2007_test.svm', dtype=np.float64, multilabel=True)

In [8]:
type(X_train),type(y_train)

(scipy.sparse.csr.csr_matrix, list)

In [6]:
X_train.ndim

2

In [7]:
X_train.shape

(21519, 30438)

In [10]:
y_train

[(1.0, 5.0, 6.0),
 (18.0,),
 (1.0, 12.0, 13.0, 18.0, 20.0),
 (17.0,),
 (11.0,),
 (1.0,),
 (2.0, 4.0),
 (5.0,),
 (11.0,),
 (4.0, 5.0),
 (11.0,),
 (1.0, 5.0),
 (1.0,),
 (1.0,),
 (6.0, 20.0),
 (1.0,),
 (6.0, 9.0, 11.0),
 (1.0, 5.0, 8.0, 18.0),
 (5.0,),
 (1.0, 18.0),
 (1.0, 5.0),
 (1.0, 9.0, 17.0),
 (1.0, 5.0, 17.0),
 (1.0, 5.0, 6.0, 11.0),
 (11.0,),
 (1.0, 5.0),
 (11.0,),
 (4.0, 5.0),
 (1.0, 5.0, 6.0),
 (1.0,),
 (1.0, 5.0, 7.0),
 (1.0, 5.0, 7.0, 11.0),
 (1.0, 5.0, 7.0, 11.0),
 (0.0, 1.0),
 (1.0,),
 (1.0, 10.0),
 (1.0, 5.0, 7.0, 11.0),
 (4.0,),
 (1.0, 11.0),
 (13.0,),
 (11.0,),
 (7.0,),
 (1.0,),
 (1.0, 5.0),
 (4.0,),
 (1.0, 4.0, 5.0),
 (11.0,),
 (5.0,),
 (1.0, 11.0),
 (1.0, 7.0, 13.0),
 (1.0, 12.0, 13.0),
 (5.0, 9.0, 18.0),
 (1.0, 5.0, 7.0, 12.0, 13.0),
 (5.0, 6.0, 11.0, 18.0),
 (1.0, 11.0, 12.0, 13.0),
 (4.0, 5.0),
 (0.0, 1.0, 18.0),
 (1.0, 5.0, 13.0),
 (18.0,),
 (5.0,),
 (5.0, 6.0),
 (1.0, 5.0, 7.0),
 (18.0, 19.0),
 (1.0, 21.0),
 (2.0, 5.0),
 (0.0, 1.0),
 (1.0, 11.0),
 (1.0,),
 (1.0,),
 

In [11]:
# convert multi labels to binary matrix
mb = MultiLabelBinarizer()
y_train = mb.fit_transform(y_train)
y_test = mb.fit_transform(y_test)

In [29]:
y_train,y_train.shape

(array([[0, 1, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 1, 0, ..., 0, 1, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]]), (21519, 22))

In [23]:
# LR + OvR
clf = OneVsRestClassifier(LogisticRegression(), n_jobs=10)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

In [30]:
y_pred

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [24]:
# multilabel classification metrics
loss = metrics.hamming_loss(y_test, y_pred)
prf = metrics.precision_recall_fscore_support(y_test, y_pred, average='samples')

/home/ian/installed/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)


In [25]:
loss,prf

(0.056983570336686065,
 (0.625062408741934, 0.5584817573584, 0.5561701820074011, None))

### scikit-multilearn
pip install scikit-multilearn  
Scikit-multilearn is a BSD-licensed library for multi-label classification that is built on top of the well-known scikit-learn ecosystem.

In [26]:

"""
ML-KNN for multilabel classification
"""
from skmultilearn.adapt import MLkNN

clf = MLkNN(k=15)
clf.fit(X_train, y_train)
y_pred1 = clf.predict(X_test)

In [32]:
y_pred1.toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       [1, 1, 0, ..., 0, 0, 0],
       ...,
       [0, 1, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [27]:
# multilabel classification metrics
loss1 = metrics.hamming_loss(y_test, y_pred1)
prf1 = metrics.precision_recall_fscore_support(y_test, y_pred1, average='samples')
loss1,prf1

/home/ian/installed/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)


(0.06524336197926703,
 (0.620375865479723, 0.6535469809420635, 0.5976903593478414, None))

AdaBoost.MH算法 Spark实现见sparkboost

In [ ]:
// AdaBoost.MH for multilabel classification
val labels0Based = true
val binaryProblem = false

val learner = new AdaBoostMHLearner(sc)
learner.setNumIterations(params.numIterations) // 500 iter
learner.setNumDocumentsPartitions(params.numDocumentsPartitions)
learner.setNumFeaturesPartitions(params.numFeaturesPartitions)
learner.setNumLabelsPartitions(params.numLabelsPartitions)
val classifier = learner.buildModel(params.input, labels0Based, binaryProblem)

val testPath = "./tmc2007_test.svm"
val numRows = DataUtils.getNumRowsFromLibSvmFile(sc, testPath)
val testRdd = DataUtils.loadLibSvmFileFormatDataAsList(sc, testPath, labels0Based, binaryProblem, 0, numRows, -1);
val results = classifier.classifyWithResults(sc, testRdd, 20)

val predAndLabels = sc.parallelize(predLabels.zip(goldLabels)
  .map(t => {
    (t._1.map(e => e.toDouble), t._2.map(e => e.toDouble))
  }))
val metrics = new MultilabelMetrics(predAndLabels)